In [31]:
import numpy as np
import random
import itertools
from __future__ import division

def check_win(board):
    global endgame
    global win
    global lose
    for w in wincombos:
        w = list(w)
        if w.count('O') == 3:
            endgame = True
            lose = True
        
        if w.count('X') == 3:
            endgame = True
            win = True


def chk_vac(board,x,y):
    if board[x,y]=='':
        return True
    return False


def make_move(board, player, marker, smart=False):
    
    new_board = board
    
    if marker == 'X':
        order = 1
    else:
        order = 2
        
    if player == 'Human':
        vac = False
        while not vac:
            x = input('Player row input:\t')
            y = input('Player column input:\t')
            if (x in range(3)) & (y in range(3)):
                if (chk_vac(new_board,x,y)):
                    new_board[x,y] = marker
                    vac = True
                else:
                    print ('Sorry position occupied. Please choose again')
            else:
                print ('Sorry invalid position. Please choose again')
                
    else:
        
        if smart:
            fb = flat_board(board)
            poss_b = gen_possible_boards(fb,marker=marker)
            #print poss_b
            picked_move = pick_optimum_move(poss_b,win_rate_dict,order)
            row, column = make_smart_move(fb,picked_move)
            new_board[row,column] = marker
        else:
            vac = False
            while not vac:
                x = np.random.randint(0,3)
                y = np.random.randint(0,3)
                if chk_vac(new_board,x,y):
                    new_board[x,y] = marker
                    vac = True
    
    check_win(new_board)
    return new_board


def flat_board(board):
    temp_board = board.flatten()
    fb = ''.join(['1' if i=='X' else '2' if i == 'O' else '0' for i in temp_board])
    return fb


def gen_possible_boards(fb,marker):
    if marker == 'X':
        num = '1'
    else:
        num = '2'
        
    poss_b = []
    for i in range(9):
        if fb[i] == '0':
            pb = fb[:i]+num+fb[i+1:]
            poss_b.append(pb)
    return poss_b


def pick_optimum_move(poss_b,win_rate_dict,order):
    all_moves = []
    reverse = True
    for i in poss_b:
        try:
            all_moves.append((i,win_rate_dict[i]))
        except:
            pass
    if order == 2:
        reverse = False
    all_moves = sorted(all_moves, key=lambda x:x[1], reverse=reverse)
    #print all_moves
    picked_move = all_moves[0][0]
    return picked_move


def make_smart_move(fb,picked_move):
    fb_array = np.array([int(i) for i in fb])
    pb_array = np.array([int(i) for i in picked_move])
    board_diff = fb_array - pb_array
    target_index = np.nonzero(board_diff)[0][0]
    row = int(np.floor(target_index/3))
    column = target_index%3
    return row, column



def assign_players():
    valid_players = False
    while not valid_players:
        human_players = input('Would you like a 1 player game or 2 players game?')
        if human_players == 1:
            print 'OK'
            return 'Computer','Human'
        elif human_players ==2:
            print 'OK'
            return 'Human','Human'
        elif human_players ==0:
            print 'OK'
            return 'Computer','Computer'
        else:
            print 'Invalid input. Please choose 0, 1 or 2'

    

def decide_order(p1,p2):
    x = np.random.randint(0,2)
    if x==0:
        return p1,p2
    return p2,p1




def play_game(train=False, smart=None):
    "Step 1"
    #global board
    global wincombos
    global endgame
    global win
    global lose
    
    p1_smart = False
    p2_smart = False
    
    if smart == 'p1':
        p1_smart = True
    elif smart == 'p2':
        p2_smart = True
    elif smart == 'all':
        p1_smart = True
        p2_smart = True
    
    p1_marker = 'X'
    p2_marker = 'O'
    endgame = False
    win = False
    lose = False
    
    if not train:
        p1, p2 = assign_players()
        p1,p2 = decide_order(p1,p2)
        if p1 == "Computer":
            print 'Computer goes first!'
        else:
            print 'Player goes first'
    else:
        p1, p2 = 'Computer','Computer'
    
    
    board_list = []
    board = np.chararray((3,3))
    board[:] = ''
    #print board
    wincombos = [board[0], board[1], board[2], board[:,0], board[:,1], board[:,2], board.diagonal(), np.fliplr(board).diagonal()]
    empty_space = 9
    while empty_space>0:
        if not train:
            print (board)
        board = make_move(board,p1,p1_marker,smart = p1_smart)
        board_list.append(flat_board(board))
        check_win(board)
        empty_space -= 1
        
        if endgame or empty_space==0:
            break
        if not train:
            print (board)
        board = make_move(board,p2,p2_marker,smart = p2_smart)
        board_list.append(flat_board(board))
        check_win(board)
        empty_space -= 1        
        if endgame:
            break 
    if win:
        if not train:
            print (board)
            print 'X wins'
            return 'End game'
        return 1,board_list
    elif lose:
        if not train:
            print (board)
            print 'O wins'
            return 'End game'
        return 2,board_list
    else:
        if not train:
            print (board)
            print 'tie'
            return 'End game'
        return 3,board_list
        
        
def gen_winrate(iterations=5):
    records = {1:[],2:[],3:[]}
    for i in range(iterations):
        result,boards = play_game(train=True)
        records[result] += boards
    return records


def get_win_rate_dict(result_records):
    unique_list = np.unique(np.array(list(itertools.chain.from_iterable(result_records.values()))))
    position_dict = {}
    for ui in unique_list:
        ui1 = result_records[1].count(ui)
        ui2 = result_records[2].count(ui)
        ui3 = result_records[3].count(ui)
        win_rate = ui1/np.sum([ui1,ui2,ui3])
        position_dict[ui] = win_rate
    return position_dict

In [32]:
result_records = gen_winrate(25000) # Number of training cycles required
win_rate_dict = get_win_rate_dict(result_records)

In [35]:
play_game(smart='all')

Would you like a 1 player game or 2 players game?1
OK
Computer goes first!
[['' '' '']
 ['' '' '']
 ['' '' '']]
[['' '' '']
 ['' 'X' '']
 ['' '' '']]
Player row input:	2
Player column input:	2
[['' '' '']
 ['' 'X' '']
 ['' '' 'O']]
[['' 'X' '']
 ['' 'X' '']
 ['' '' 'O']]
Player row input:	2
Player column input:	1
[['' 'X' '']
 ['' 'X' '']
 ['' 'O' 'O']]
[['' 'X' '']
 ['' 'X' '']
 ['X' 'O' 'O']]
Player row input:	0
Player column input:	2
[['' 'X' 'O']
 ['' 'X' '']
 ['X' 'O' 'O']]
[['' 'X' 'O']
 ['X' 'X' '']
 ['X' 'O' 'O']]
Player row input:	1
Player column input:	2
[['' 'X' 'O']
 ['X' 'X' 'O']
 ['X' 'O' 'O']]
O wins


'End game'

#### AI idea

By looking at certain board positions, computer calculates winning probabilities. Repeat computer vs computer game to collect data for different board positions. Should be a classification problem with predict proba. rank the 


#### Game Logic

1. Initiate empty board
2. random computer 1 make move
3. log board position
4. random computer 2 make move 
5. log board position
6. repeat 2-5 until winner emerges or board full

Things to think about:
- board positions can be flattern in order to be stored. as a 1D array
- No need to log empty board!?

#### Data structure

- labels = win (1) or lose (0)
- features = board positions